In [1]:
import healpy as hp
from astropy import units as u
from astropy.coordinates import SkyCoord
from numpy import *
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from astropy.io import fits
from scipy.interpolate import griddata

In [2]:
nside=64

with fits.open('/Users/nicholebarry/MWA/data/EDA2/EDA2_159MHz_I_wPriori_mono.fits') as hdu:
    data = hdu[0].data

with fits.open('/Users/nicholebarry/MWA/data/EDA2/EDA2_159-408MHz_SI_wPriori.fits') as hdu:
    si = hdu[0].data

In [9]:
pix_inds = arange(hp.nside2npix(nside))
l, b = hp.pix2ang(nside,pix_inds,lonlat=True)
cel_coords = SkyCoord(l*u.deg, b*u.deg, frame='icrs')
ra = cel_coords.icrs.ra.value
dec = cel_coords.icrs.dec.value
 
l = cel_coords.galactic.l.value
b = cel_coords.galactic.b.value
shift_inds = np.argwhere((l > 180))
l[shift_inds] = l[shift_inds] - 360
#galactic_plane_inds = np.argwhere((b < 25) & (b > -25))
#ra_subset = ra[galactic_plane_inds]
#dec_subset = dec[galactic_plane_inds]
#data_subset = data[galactic_plane_inds]
#b_subset = b[galactic_plane_inds]
#l_subset = l[galactic_plane_inds]
print(hp.nside2pixarea(2048,degrees=True)*60*60)

k_boltz = 1.38064852e-23
vel_c = 299792458.0
freq=182e6
Jystr = (1e26*2*freq**2*k_boltz) / vel_c**2
conversion = hp.nside2pixarea(nside) * Jystr
#conversion = Jystr 

fluxes = data*conversion
print(np.abs(fluxes).max(),np.abs(fluxes).mean())

2.950641721445508
2185.9775 47.07638


In [32]:
# target grid to interpolate to
step_size=0.05
yi = np.arange(-90,90,step_size)
xi = np.arange(0,360,step_size)
xi,yi = np.meshgrid(xi,yi)

# interpolate
fluxes_grid = griddata((l,b),fluxes,(xi,yi),method='linear')
pix_area = (step_size*np.pi/180)**2
fluxes_grid = fluxes_grid * pix_area

#put into array
fits_arr=np.array([fluxes_grid,xi,yi])
fits_arr.shape

8550215.0 184134.17


(3, 18000, 36000)

In [7]:
# target grid to interpolate to -- shifted
step_size=0.05
yi = np.arange(-90,90,step_size)
xi = np.arange(0,360,step_size) - 180
xi,yi = np.meshgrid(xi,yi)

# interpolate
fluxes_grid = griddata((l,b),fluxes,(xi,yi),method='linear')
pix_area = (step_size*np.pi/180)**2
fluxes_grid = fluxes_grid * pix_area

#put into array
fits_arr=np.array([fluxes_grid,xi,yi])
fits_arr.shape

(3, 3600, 7200)

In [10]:
hdu = fits.PrimaryHDU(fits_arr)
hdu.writeto('/Users/nicholebarry/MWA/data/EDA2/fluxesJyperpix_londeg_latdeg_shift.fits')

In [9]:
fig = plt.figure(figsize=(10,10))
vmin = -100#-0.1
vmax = 100#0.1

#hp.mollview(fluxes, sub=(2,1,1), fig=fig,title='Galactic (Jy / pixel)',min=vmin,max=vmax)
ax2 = fig.add_subplot(2,1,2)
ax2.scatter(l,b,c=fluxes,marker='.',vmin=vmin,vmax=vmax)
#ax2.scatter(l_subset,b_subset,c=fluxes,marker='.',vmin=vmin,vmax=vmax)
ax2.plot(266.416833333,-29.0078055556,'ro',mfc='none',label="Where gal centre should be")
ax2.set_xlabel('RA (deg)')
ax2.set_ylabel('Dec (deg)')
ax2.legend(loc='upper left')

fig.savefig('/Users/nicholebarry/MWA/data/EDA2/EDA2_n64_gal_subset_shift.png',bbox_inches='tight')
plt.close()

In [ ]:
longitude = 0 * u.deg
latitude = -90 * u.deg
rot_custom = hp.Rotator(rot=[longitude.to_value(u.deg), latitude.to_value(u.deg)], inv=True)
fluxes_rotated_alms = rot_custom.rotate_map_alms(fluxes)

hp.mollview(fluxes_rotated_alms, sub=(2,1,1), fig=fig,title='Galactic (Jy / pixel)',min=vmin,max=vmax)
ax2 = fig.add_subplot(2,1,2)
ax2.scatter(l,b,c=fluxes_rotated_alms,marker='.',vmin=vmin,vmax=vmax)
ax2.set_xlabel('Lat (deg)')
ax2.set_ylabel('90deg rotated Lon (deg)')
fig.savefig('falselon_pygdsm_nomono_nogalaxy_n2048.png',bbox_inches='tight')
plt.close()

center_patch_inds = np.argwhere((l > 170) & (l < 190) & (b > -10) & (b < 10))
center_patch = fluxes_rotated_alms[center_patch_inds]
rotated_lonlat = rot_custom(l,b,lonlat=True)
center_patch_latlon = rotated_lonlat[:,center_patch_inds]



hdu = fits.PrimaryHDU(l[center_patch_inds])
hdu.writeto('center_patch_lat_1024.fits', clobber=True)
hdu = fits.PrimaryHDU(b[center_patch_inds])
hdu.writeto('center_patch_lon_1024.fits', clobber=True)